## Variable Preprocessing

In [1]:
import ast, json, random, argparse
import os, sys, datetime, shutil
import statistics, smtplib, ssl, re, string
import datetime, math, pickle
import pdb
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px
#
from plotly import tools
from plotly.subplots import make_subplots
from datetime import date, timedelta

In [2]:
fich='./DatosGranadilla_5minutos.xlsx'
sheet_to_df_map = pd.read_excel(fich, sheet_name=None)

In [3]:
sheet_to_df_map.keys()

dict_keys(['TG5', 'TG6', 'CALDERA 5', 'CALDERA 6', 'TURBINA VAPOR 4_CICLO AGUAVAPOR'])

In [4]:
sheet_to_df_map['TG5']

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SISTEMA,TURBINA DE GAS 5 - COMBUSTIBLE,NaN,TURBINA DE GAS 5 - POTENCIA ACTIVA,NaN,TURBINA DE GAS 5 - POTENCIA REACTIVA,NaN,TURBINA DE GAS 5 - CHIMENEA CICLO ABIERTO - TE...,NaN,TURBINA DE GAS 5 - CHIMENEA CICLO ABIERTO - PR...,...,TURBINA DE GAS 5 - TEMPERATURA DE ESCAPE DE TU...,NaN,TURBINA DE GAS 5 - PRIMER NIVEL DE DISPERSION ...,NaN,TURBINA DE GAS 5 - NIVEL DE DISPERSIÓN MAXIMO ...,NaN,TURBINA DE GAS 5 - FACTOR DE POTENCIA DE BARRA...,NaN,TURBINA DE GAS 5 - VELOCIDAD DE ROTOR DE TURBINA,NaN
2,NaN,kg/s,NaN,MW,NaN,Mvar,NaN,ºC,NaN,bara,...,ºC,NaN,ºC,NaN,ºC,NaN,NaN,NaN,rpm,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51945,2023-07-31 22:39:39.770000,3.933686,100,49.940753,100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51946,2023-07-31 22:44:39.770000,3.907359,100,49.28498,100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51947,2023-07-31 22:49:39.770000,3.851733,100,48.221528,100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51948,2023-07-31 22:54:39.770000,3.715625,100,47.261545,100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
alld = {}
for ik in sheet_to_df_map.keys():
    alld[ik] = {}
    dat=sheet_to_df_map[ik].dropna(how='all').copy()
    # Extracting Data and Quality values
    lcls = list(range(1,len(dat.iloc[0,:]),2))
    ncls = ['time']
    unts = ['ISO_format']
    dets = ['DateTime']
    addr = ['TimeStamp']
    for j in lcls:
        tmp = dat.iloc[0,j].strip().split('-')
        sys = tmp[0].strip()
        ncls.append(':'.join([x.strip().replace(' ','_') for x in tmp[1:]]))
        unts.append(dat.iloc[1,j])
        dets.append(dat.iloc[2,j])
        addr.append(dat.iloc[3,j])
    unts = ['n.d.' if isinstance(x,float) else x for x in unts]
    fcls = []
    for var,unt in zip(ncls,unts):
        fcls.append(var+':'+unt)
    dtset = dat.iloc[7:,[0]+lcls]  # dataframe for pure data
    dtset.columns=fcls
    qdtset = dat.iloc[7:,[j+1 for j in lcls]] # dataframe for data quality
    txtcls = ['Q:'+j.split(':')[0] for j in fcls[1:]]
    qdtset.columns= txtcls
    #
    alld[ik]['Title'] = sys
    alld[ik]['Dat']   = dtset
    alld[ik]['Qdat']  = qdtset
    alld[ik]['desc']  = dets
    alld[ik]['addr']  = addr
del sheet_to_df_map
with open('data_5m.pkl', 'wb') as handle:
    pickle.dump(alld, handle, protocol=pickle.HIGHEST_PROTOCOL)
#

In [6]:
for ik in alld.keys():
    print(alld[ik]['Title']+ " :"+ik)
    print(alld[ik]['Dat'].columns)

TURBINA DE GAS 5 :TG5
Index(['time:ISO_format', 'COMBUSTIBLE:kg/s', 'POTENCIA_ACTIVA:MW',
       'POTENCIA_REACTIVA:Mvar',
       'CHIMENEA_CICLO_ABIERTO:TEMPERATURA_ESCAPE:ºC',
       'CHIMENEA_CICLO_ABIERTO:PRESION:bara',
       'CONTADOR_DE_AGUA_DE_INYECCION:kg', 'POSICIÓN_IGV_ENTRADA_COMPRESOR:º',
       'PRESION_DIFERENCIAL_FILTROS_ENTRADA_AIRE_ANTES_COMPRESOR:barg',
       'TEMPERATURA_ENTRADA_AIRE_ANTES_DE_COMPRESOR:ºC',
       'PRESION_DE_DESCARGA_COMPRESOR:barg',
       'TEMPERATURA_DE_DESCARGA_COMPRESOR:ºC',
       'TEMPERATURA_DE_REFERENCIA_CONTROL_COMBUSTION:ºC',
       'TEMPERATURA_DE_ESCAPE_DE_TURBINA:ºC',
       'PRIMER_NIVEL_DE_DISPERSION_DE_TEMPERATURAS_DE_ESCAPE:ºC',
       'NIVEL_DE_DISPERSIÓN_MAXIMO_PERMITIDO:ºC',
       'FACTOR_DE_POTENCIA_DE_BARRAS_DE_ALIMENTACION_MT:n.d.',
       'VELOCIDAD_DE_ROTOR_DE_TURBINA:rpm'],
      dtype='object')
TURBINA DE GAS 6 :TG6
Index(['time:ISO_format', 'COMBUSTIBLE:kg/s', 'POTENCIA_ACTIVA:MW',
       'POTENCIA_REACTIVA:Mvar',
   

In [7]:
kvr1=alld['TG5']['Dat'][['time:ISO_format','CHIMENEA_CICLO_ABIERTO:TEMPERATURA_ESCAPE:ºC']]
kvr1

,time:ISO_format,CHIMENEA_CICLO_ABIERTO:TEMPERATURA_ESCAPE:ºC
10,2023-02-01 13:44:39.770000,68.469776
11,2023-02-01 13:49:39.770000,68.579295
12,2023-02-01 13:54:39.770000,68.863646
13,2023-02-01 13:59:39.770000,68.510283
14,2023-02-01 14:04:39.770000,68.115376
...,...,...
51945,2023-07-31 22:39:39.770000,NaN
51946,2023-07-31 22:44:39.770000,NaN
51947,2023-07-31 22:49:39.770000,NaN
51948,2023-07-31 22:54:39.770000,NaN


In [11]:
r, _ = np.where(kvr1.isna())
r

array([51933, 51934, 51935, 51936, 51937, 51938, 51939])

In [12]:
r, _ = np.where(alld['TG5']['Dat'].isna())
r

array([51932, 51932, 51932, 51932, 51932, 51932, 51932, 51932, 51932,
       51932, 51932, 51933, 51933, 51933, 51933, 51933, 51933, 51933,
       51933, 51933, 51933, 51933, 51933, 51933, 51933, 51933, 51934,
       51934, 51934, 51934, 51934, 51934, 51934, 51934, 51934, 51934,
       51934, 51934, 51934, 51934, 51934, 51935, 51935, 51935, 51935,
       51935, 51935, 51935, 51935, 51935, 51935, 51935, 51935, 51935,
       51935, 51935, 51936, 51936, 51936, 51936, 51936, 51936, 51936,
       51936, 51936, 51936, 51936, 51936, 51936, 51936, 51936, 51937,
       51937, 51937, 51937, 51937, 51937, 51937, 51937, 51937, 51937,
       51937, 51937, 51937, 51937, 51937, 51938, 51938, 51938, 51938,
       51938, 51938, 51938, 51938, 51938, 51938, 51938, 51938, 51938,
       51938, 51938, 51939, 51939, 51939, 51939, 51939, 51939, 51939,
       51939, 51939, 51939, 51939, 51939, 51939, 51939, 51939, 51939])

In [15]:
alld['TG5']['Dat'].loc[51945]

time:ISO_format                                                  2023-07-31 22:39:39.770000
COMBUSTIBLE:kg/s                                                                   3.933686
POTENCIA_ACTIVA:MW                                                                49.940753
POTENCIA_REACTIVA:Mvar                                                                  NaN
CHIMENEA_CICLO_ABIERTO:TEMPERATURA_ESCAPE:ºC                                            NaN
CHIMENEA_CICLO_ABIERTO:PRESION:bara                                                     NaN
CONTADOR_DE_AGUA_DE_INYECCION:kg                                                        NaN
POSICIÓN_IGV_ENTRADA_COMPRESOR:º                                                        NaN
PRESION_DIFERENCIAL_FILTROS_ENTRADA_AIRE_ANTES_COMPRESOR:barg                           NaN
TEMPERATURA_ENTRADA_AIRE_ANTES_DE_COMPRESOR:ºC                                          NaN
PRESION_DE_DESCARGA_COMPRESOR:barg                                              

In [16]:
alld['TG5']['Dat'].loc[51948]

time:ISO_format                                                  2023-07-31 22:54:39.770000
COMBUSTIBLE:kg/s                                                                   3.715625
POTENCIA_ACTIVA:MW                                                                47.261545
POTENCIA_REACTIVA:Mvar                                                                  NaN
CHIMENEA_CICLO_ABIERTO:TEMPERATURA_ESCAPE:ºC                                            NaN
CHIMENEA_CICLO_ABIERTO:PRESION:bara                                                     NaN
CONTADOR_DE_AGUA_DE_INYECCION:kg                                                        NaN
POSICIÓN_IGV_ENTRADA_COMPRESOR:º                                                        NaN
PRESION_DIFERENCIAL_FILTROS_ENTRADA_AIRE_ANTES_COMPRESOR:barg                           NaN
TEMPERATURA_ENTRADA_AIRE_ANTES_DE_COMPRESOR:ºC                                          NaN
PRESION_DE_DESCARGA_COMPRESOR:barg                                              

In [17]:
qkvr=alld['TG5']['Qdat'].dropna()
idx = qkvr.iloc[:,1] != 100
qkvr.loc[idx,'Q:COMBUSTIBLE']

36568                                93.520386
38525                                52.127157
38526    [-11059] No Good Data For Calculation
38527    [-11059] No Good Data For Calculation
38528    [-11059] No Good Data For Calculation
38529    [-11059] No Good Data For Calculation
38530    [-11059] No Good Data For Calculation
38531                                  7.92362
38532                                      100
Name: Q:COMBUSTIBLE, dtype: object

In [18]:
jdx = qkvr.iloc[:,2] != 100
qkvr.loc[jdx]

,Q:COMBUSTIBLE,Q:POTENCIA_ACTIVA,Q:POTENCIA_REACTIVA,Q:CHIMENEA_CICLO_ABIERTO,Q:CHIMENEA_CICLO_ABIERTO,Q:CONTADOR_DE_AGUA_DE_INYECCION,Q:POSICIÓN_IGV_ENTRADA_COMPRESOR,Q:PRESION_DIFERENCIAL_FILTROS_ENTRADA_AIRE_ANTES_COMPRESOR,Q:TEMPERATURA_ENTRADA_AIRE_ANTES_DE_COMPRESOR,Q:PRESION_DE_DESCARGA_COMPRESOR,Q:TEMPERATURA_DE_DESCARGA_COMPRESOR,Q:TEMPERATURA_DE_REFERENCIA_CONTROL_COMBUSTION,Q:TEMPERATURA_DE_ESCAPE_DE_TURBINA,Q:PRIMER_NIVEL_DE_DISPERSION_DE_TEMPERATURAS_DE_ESCAPE,Q:NIVEL_DE_DISPERSIÓN_MAXIMO_PERMITIDO,Q:FACTOR_DE_POTENCIA_DE_BARRAS_DE_ALIMENTACION_MT,Q:VELOCIDAD_DE_ROTOR_DE_TURBINA
36562,100,100,97.794047,65.68457,52.853333,74.427409,75.427409,77.373444,99.666667,100,100,100,100,100,100,100,100
38519,100,100,92.59756,81.965678,57.096537,45.961243,35.482295,7.055486,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation
38520,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation
38521,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation
38522,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation
38523,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation
38524,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,20.833832,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,100,100,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,15.966049
38525,52.127157,52.866018,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,38.72495,33.844523,100,65.413823,68.984507,100,100,85.814295,93.250966,98.355906,45.523554,100
38526,[-11059] No Good Data For Calculation,[-11059] No Good Data For Calculation,66.171967,27.610764,47.646571,100,100,100,100,100,100,100,100,100,100,100,100


In [56]:
alld['TG6']['addr'] = alld['TG6']['desc']
alld['TG6']['addr'] = list(map(lambda x: x.replace('DateTime', 'TimeStamp'), alld['TG6']['addr']))

In [57]:
# Recovering the name of the variables (without path).
def extractv(txt):
    j = txt.split('\\')
    k = len(j)
    return(j[k-1])
#
for ik in alld.keys():
    lvr = []
    for j in alld[ik]['addr']:
        lvr.append(extractv(j))
    #
    alld[ik]['addrv'] = lvr
#

In [58]:
with open('data_5m.pkl', 'wb') as handle:
    pickle.dump(alld, handle, protocol=pickle.HIGHEST_PROTOCOL)
#

## Loading CSV (Second dataset)

In [20]:
fich       = './DatosICAI_1923.csv'
data_sheet = pd.read_csv(fich, sep=';')

In [21]:
data_sheet

,index,GRLL0221FQLM1,GRLL0221dwattout2,GRLL0221dvar,GRLL0221TT_MES_T BP1,GRLL0221TT_MES_P_BP,GRLL0221TOTH2O,GRLL0221csgv,GRLL0221afpcs,GRLL0221CTIM,...,GRLL0220PAB32CT007JT01A,GRLL0220PAB31CT007JT01A,GRLL0220LCR01CL002XW01,GRLL0220DWATT,GRLL0220DPF,GRLL0220TNH_RPM,GRLL0221TTAAC586,GRLL0222TTAAC586,GRLL-MEDAS-EMI-EMISARIO_TEMP_AMB_UA,GRLL-MEDAS-EMI-EMISARIO_PRESION_UA
0,12/09/2019 0:00,0.000000,0.041784,-0.107773,89.253120,1014.375000,8.368689e+08,26.309298,0.540243,30.168663,...,26.300991,26.216450,4843.874512,0.007275,1.0,5.622362,0.0,0.0,22.973090,1018.348846
1,12/09/2019 0:10,NaN,0.041001,-0.107773,80.396873,1014.450012,8.368689e+08,26.309298,NaN,30.549444,...,26.298168,NaN,4843.191895,0.007275,1.0,NaN,0.0,0.0,22.873143,1018.351705
2,12/09/2019 0:20,0.000000,NaN,NaN,NaN,1014.000000,NaN,NaN,0.540243,30.170935,...,26.306322,26.249367,NaN,NaN,NaN,5.622158,0.0,0.0,22.959286,1018.251535
3,12/09/2019 0:30,0.000000,0.048415,-0.107773,73.693748,NaN,8.368689e+08,26.309298,0.540243,NaN,...,NaN,26.250708,4843.742676,0.007275,1.0,5.614914,NaN,NaN,22.843907,1018.336135
4,12/09/2019 0:40,0.000000,0.040125,-0.107773,NaN,1014.224976,8.368689e+08,26.309298,NaN,29.792425,...,26.313633,NaN,4843.610352,0.007275,1.0,5.619725,0.0,0.0,22.698326,1018.343745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210523,12/09/2023 23:10,4.006058,49.758185,2.040973,77.404165,1012.959998,1.303287e+09,61.456192,NaN,27.572779,...,30.345870,30.023593,4728.102539,56.814270,NaN,3000.247196,NaN,NaN,24.430941,1017.718819
210524,12/09/2023 23:20,3.965449,50.421466,1.588644,77.122498,1012.969442,1.303289e+09,NaN,111.006210,27.531142,...,30.314618,29.982331,NaN,57.220772,NaN,3000.565691,NaN,NaN,24.261675,1017.725038
210525,12/09/2023 23:30,3.865254,48.186684,1.129685,75.186716,1013.046439,1.303292e+09,59.942158,NaN,27.628768,...,30.230586,29.893803,4718.394531,56.201890,NaN,3000.413990,NaN,NaN,24.264202,1017.787900
210526,12/09/2023 23:40,3.832138,47.337365,0.825578,75.429543,1013.093742,1.303294e+09,NaN,106.501289,27.615380,...,30.137141,29.800999,NaN,55.514882,NaN,2999.914132,NaN,NaN,24.498215,1017.864636


In [64]:
print(alld.keys())
print(alld['TG5'].keys())

dict_keys(['TG5', 'TG6', 'CALDERA 5', 'CALDERA 6', 'TURBINA VAPOR 4_CICLO AGUAVAPOR'])
dict_keys(['Title', 'Dat', 'Qdat', 'desc', 'addr', 'addrv'])


In [49]:
data_sheet = data_sheet.rename(columns={"index": "TimeStamp"})

In [65]:
data_sheet.columns

Index(['TimeStamp', 'GRLL0221FQLM1', 'GRLL0221dwattout2', 'GRLL0221dvar',
       'GRLL0221TT_MES_T BP1', 'GRLL0221TT_MES_P_BP', 'GRLL0221TOTH2O',
       'GRLL0221csgv', 'GRLL0221afpcs', 'GRLL0221CTIM', 'GRLL0221CPD',
       'GRLL0221CTDA', 'GRLL0221TTRF1', 'GRLL0221TTXM', 'GRLL0221TTXSP1',
       'GRLL0221TTXSPL', 'GRLL0221BBA10GS511ET16R', 'GRLL0221tnh_v',
       'GRLL0222FQLM1', 'GRLL0222dwatt', 'GRLL0222dvar',
       'GRLL0222TT_MES_T BP2', 'GRLL0222TT_MES_P_BP', 'GRLL0222TOTH2O',
       'GRLL0222csgv', 'GRLL0222afpcs', 'GRLL0222CTIM', 'GRLL0222CPD',
       'GRLL0222CTDA', 'GRLL0222TTRF1', 'GRLL0222TTXM', 'GRLL0222TTXSP1',
       'GRLL0222TTXSPL', 'GRLL0222BBA10GS521ET16R', 'GRLL0222tnh_v',
       'GRLL0221LAP50CF001XZ01A', 'GRLL0221LCA90CF001XZ01_A',
       'GRLL0221LBA10CF001XZ01A', 'GRLL0221LBD10CF001XZ01A',
       'GRLL0221LAE01CF001XZ01A', 'GRLL0220cv1_pos', 'GRLL0220acv1_pos',
       'GRLL0220IP_P', 'GRLL0220TT_IS', 'GRLL0221HNA20CT401JT01A',
       'GRLL0221TT_MES_T HRSG1', '

In [67]:
vTG5 = data_sheet[alld['TG5']['addrv']]

In [68]:
vTG6 = data_sheet[alld['TG6']['addrv']]

In [69]:
vCD5 = data_sheet[alld['CALDERA 5']['addrv']]

In [70]:
vCD6 = data_sheet[alld['CALDERA 6']['addrv']]

In [71]:
vTV4 = data_sheet[alld['TURBINA VAPOR 4_CICLO AGUAVAPOR']['addrv']]

In [73]:
usedcols = vTG5.columns.tolist() + vTG6.columns.tolist() + vCD5.columns.tolist() + vCD6.columns.tolist() + vTV4.columns.tolist()

In [78]:
usedcols = list(set(usedcols))
list(set(data_sheet.columns.tolist())-set(usedcols))

['GRLL0221TTAAC586',
 'GRLL-MEDAS-EMI-EMISARIO_PRESION_UA',
 'GRLL-MEDAS-EMI-EMISARIO_TEMP_AMB_UA',
 'GRLL0222TTAAC586']

In [84]:
# Adding the first var to TG5 and the last one to TG6
vTG5 = data_sheet[alld['TG5']['addrv']+['GRLL-MEDAS-EMI-EMISARIO_PRESION_UA',\
                                                                    'GRLL-MEDAS-EMI-EMISARIO_TEMP_AMB_UA']]
vTG6 = data_sheet[alld['TG6']['addrv']+['GRLL-MEDAS-EMI-EMISARIO_PRESION_UA',\
                                                                    'GRLL-MEDAS-EMI-EMISARIO_TEMP_AMB_UA']]
vCD5 = data_sheet[alld['CALDERA 5']['addrv']+['GRLL0221TTAAC586','GRLL-MEDAS-EMI-EMISARIO_PRESION_UA',\
                                                                    'GRLL-MEDAS-EMI-EMISARIO_TEMP_AMB_UA']]
vCD6 = data_sheet[alld['CALDERA 6']['addrv']+['GRLL0222TTAAC586','GRLL-MEDAS-EMI-EMISARIO_PRESION_UA',\
                                                                    'GRLL-MEDAS-EMI-EMISARIO_TEMP_AMB_UA']]
vTV4 = data_sheet[alld['TURBINA VAPOR 4_CICLO AGUAVAPOR']['addrv']+['GRLL-MEDAS-EMI-EMISARIO_PRESION_UA',\
                                                                    'GRLL-MEDAS-EMI-EMISARIO_TEMP_AMB_UA']]

In [85]:
print(vTG5.shape)
print(vTG6.shape)
print(vCD5.shape)
print(vCD6.shape)
print(vTV4.shape)

(210528, 19)
(210528, 19)
(210528, 13)
(210528, 13)
(210528, 14)


In [88]:
for j in vTG5.columns:
    print('{} => NaNs: {}'.format(j,vTG5[j].isna().sum()))

TimeStamp => NaNs: 0
GRLL0221FQLM1 => NaNs: 18536
GRLL0221dwattout2 => NaNs: 4734
GRLL0221dvar => NaNs: 18199
GRLL0221TT_MES_T BP1 => NaNs: 15802
GRLL0221TT_MES_P_BP => NaNs: 12399
GRLL0221TOTH2O => NaNs: 20121
GRLL0221csgv => NaNs: 88259
GRLL0221afpcs => NaNs: 76298
GRLL0221CTIM => NaNs: 68201
GRLL0221CPD => NaNs: 39652
GRLL0221CTDA => NaNs: 116817
GRLL0221TTRF1 => NaNs: 135151
GRLL0221TTXM => NaNs: 16331
GRLL0221TTXSP1 => NaNs: 17421
GRLL0221TTXSPL => NaNs: 43509
GRLL0221BBA10GS511ET16R => NaNs: 14872
GRLL0221tnh_v => NaNs: 10541
GRLL0221TTAAC586 => NaNs: 113813


In [89]:
vTG5.dropna().shape

(17463, 19)

In [91]:
for j in vTG6.columns:
    print('{} => NaNs: {}'.format(j,vTG6[j].isna().sum()))

TimeStamp => NaNs: 0
GRLL0222FQLM1 => NaNs: 22125
GRLL0222dwatt => NaNs: 21581
GRLL0222dvar => NaNs: 21502
GRLL0222TT_MES_T BP2 => NaNs: 18556
GRLL0222TT_MES_P_BP => NaNs: 12931
GRLL0222TOTH2O => NaNs: 22766
GRLL0222csgv => NaNs: 78468
GRLL0222afpcs => NaNs: 72094
GRLL0222CTIM => NaNs: 71342
GRLL0222CPD => NaNs: 40486
GRLL0222CTDA => NaNs: 101996
GRLL0222TTRF1 => NaNs: 135122
GRLL0222TTXM => NaNs: 19212
GRLL0222TTXSP1 => NaNs: 16492
GRLL0222TTXSPL => NaNs: 45430
GRLL0222BBA10GS521ET16R => NaNs: 6355
GRLL0222tnh_v => NaNs: 8382
GRLL0222TTAAC586 => NaNs: 113829


In [92]:
vTG6.dropna().shape

(17323, 19)